In [48]:
import json
import time
import sys
import os
from azureml.core.model import Model
import numpy as np    # we're going to use numpy to process input and output data
#import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests
import pandas as pd
import logging
import sklearn
from sklearn import preprocessing
logging.basicConfig(level=logging.DEBUG)


class TimeRNN(nn.Module):
    def __init__(self,bat_size,in_features,h_size,layer_amnt):
        super(TimeRNN,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.batch_sz = bat_size
        self.in_features = in_features
        self.h_size = h_size
        self.layer_amnt = layer_amnt
        
        self.lstm1 = nn.LSTM(input_size=self.in_features,
                             hidden_size=self.h_size,
                             num_layers=self.layer_amnt,
                             bias=True,
                             batch_first=False,
                             dropout=0,
                             bidirectional=False)
        self.fc1 = nn.Linear(in_features=1,out_features=1)
    def init_hidden(self):
        """Intialize/re-init the hidden and cell states. 
        The hidden state acts as the memory of the RNN 
        which gets passed from one unit to another. 
        h_i = f(h_i + in)

        Intializing with 0s
        """
        #print('layer size =\t', self.layer_amnt)
        #print('bat_size =\t', self.batch_sz)
        #print('hidden size =\t',self.h_size)
        return (torch.zeros(self.layer_amnt,self.batch_sz,self.h_size),
                torch.zeros(self.layer_amnt,self.batch_sz,self.h_size))
    def forward(self,x):
        x = x.unsqueeze(0)
        hidden_init = self.init_hidden()
        h0 = hidden_init[0].to(self.device)
        c0 = hidden_init[1].to(self.device)
        x,hidden = self.lstm1( x,(h0,c0))
        x = F.leaky_relu(self.fc1(x[-1].view(self.batch_sz,-1)))
        return x

class Inferencer(object):
    def __init__(self):
        pass
    
    def open_model(self,location):
        model = TimeRNN(bat_size=1,in_features=3,h_size=1,layer_amnt=1)
        model.load_state_dict(torch.load(location))
        model.eval()
        return model

    def un_normalize(self,norm_val,min_val,max_val,typelist=None):
        if(typelist):
            for idx,item in enumerate(norm_val):
                new_val = item * (max_val - min_val) + min_val
                norm_val[idx] = new_val
            return norm_val
        else:
            return norm_val * (max_val - min_val) + min_val 

    def inference(self,value, normalize_method, model,minimum_price,maximum_price):
        value = np.array(value)
        predictions = []
        for sample in value:
            sample = np.array(sample).reshape(1,-1)
            example = torch.tensor(normalize_method.transform(sample)).float()
            
            if(str(device) == 'cuda'):
                example = example.to(device)

            output = model(example)
            output_unnorm = self.un_normalize(norm_val=output.detach(),min_val=minimum_price,max_val=maximum_price)
            predictions.append(output_unnorm)
        return predictions

    def fetch_latest_BTC_JSON(self):
        """Fetch the latest JSON data"""
        API_LINK = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=SAITMI5ZUMGEKGKY'
        page = requests.get(API_LINK).json()
        return page
    def parse_alphaV_JSON(self,raw_data):
        # Remove meta data for now
        raw_data.pop('Meta Data',None)
        # Remove key name
        df = pd.DataFrame.from_dict(raw_data['Time Series (Digital Currency Daily)'],dtype=float)
        # Flip dates as columns into rows
        df = df.transpose()
        return df


def init():
    global model
    inf = Inferencer()
    model_path = Model.get_model_path(model_name='price_predictor.pt')
    #print(model_path)
    #model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model = inf.open_model(location=model_path)
    #model.eval()
    
def preprocess(input_data):
    # convert the JSON data into the tensor input
    return torch.tensor(input_data).float()

def postprocess(result):
    
    result = np.array(result).item()
    
    inf = Inferencer()
    
    raw_data = inf.fetch_latest_BTC_JSON()
    df = inf.parse_alphaV_JSON(raw_data=raw_data)
    prices = np.array(df['4a. close (USD)'].tolist())
    data_df_temp = df.drop(labels=['1a. open (USD)','1b. open (USD)','2b. high (USD)','3b. low (USD)','4a. close (USD)','4b. close (USD)','6. market cap (USD)'],axis=1)
    minmax_2 = preprocessing.MinMaxScaler()
    data_df_temp = pd.DataFrame(minmax_2.fit_transform(data_df_temp), columns=data_df_temp.columns)

    minimum_price = np.min(prices)
    maximum_price = np.max(prices)

    res = inf.un_normalize(norm_val=result,min_val=minimum_price,max_val=maximum_price)
    return res

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data = preprocess(input_data_json) 
        #result = session.run([], {input_name: input_data})
        output = model(input_data)
        res = postprocess(output.detach())
        end = time.time()     # stop timer
        return {"result": res,"time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

In [49]:
test_sample = [[8700,11080,25000]]

# test = preprocess(test_sample)
# print(test)

# test = postprocess([[0.2312]])
# print(test)

init()
res = run(test_sample)
print(res)

{'result': 11965.685513139926, 'time': 1.31931471824646}
